Considere:

$$
\begin{align*}
    \begin{bmatrix}
        1 & 2 & 3\\
        3 & 2 & 1\\
        1 & 1 & -1
    \end{bmatrix}
    \mathbf{x} &=
    \begin{bmatrix}
        1\\
        1\\
        1
    \end{bmatrix}.
\end{align*}
$$

1. Construya el sub-espacio de Krylov *no-ortonormalizado* asociado.
2. Construya el sub-espacio de Krylov *ortonormalizado* asociado.
3. Construya la descomposición de Hessemberg asociada.
4. Determine el sistema de ecuaciones *equivalente* a resolver con GMRes.

In [1]:
import numpy as np

In [13]:
A = np.array([[1.,2.,3.],[3.,2.,1.],[1.,1.,-1.]])
b = np.array([1.,1.,1.])
A


array([[ 1.,  2.,  3.],
       [ 3.,  2.,  1.],
       [ 1.,  1., -1.]])

sub-espacio de Krylov

$b_1 = b, \ b_2 = A\,b, \ b_3 = A^2b = A(A\,b) = A\,b_2$

In [3]:
b1 = b
b2 = np.dot(A,b1)
b3 = np.dot(A,b2)
print(b1,b2,b3)

[1. 1. 1.] [6. 6. 1.] [21. 31. 11.]


$q_1 = b / \lVert b \rVert$

In [4]:
q1 = b / np.linalg.norm(b)
q1

array([0.57735027, 0.57735027, 0.57735027])

La ecuación para $A\,q_1$ es:

$A\,q_1 = h_{11}\,q_1 + h_{21}\,q_2$

Multiplicamos $q_1^*$ a la ecuación:

$q_1^*(A\,q_1) = h_{11}$

Restamos $h_{11}\,q_1$:

$A\,q_1 - h_{11}\,q_1 = h_{21}\,q_2$

Aplicamos norma:

$\lVert A\,q_1 - h_{11}\,q_1 \rVert = h_{21}$

Obtenemos $q_2$:

$q_2 = (A\,q_1 - h_{11}\,q_1) / h_{21}$

In [5]:
y = np.dot(A,q1)
h11 = np.dot(q1,y)
y = y - h11*q1
h21 = np.linalg.norm(y)
q2 = y / h21

La ecuación para $A\,q_2$ es:

$A\,q_2 = h_{12}\,q_1 + h_{22}\,q_2 + h_{32}\,q_3$

Multiplicamos $q_1^*$ a la ecuación:

$q_1^*(A\,q_2) = h_{12}$

Multiplicamos $q_2^*$ a la ecuación:

$q_2^*(A\,q_2 - h_{12}\,q_1) = h_{22}$

Restamos $h_{12}\,q_1 + h_{22}\,q_2$:

$A\,q_2 - h_{12}\,q_1 - h_{22}\,q_2 = h_{32}\,q_3$

Aplicamos norma:

$\lVert A\,q_2 - h_{12}\,q_1 - h_{22}\,q_2 \rVert = h_{32}$

Obtenemos $q_3$:

$q_3 = (A\,q_2 - h_{12}\,q_1 - h_{22}\,q_2) / h_{32}$

In [6]:
y = np.dot(A,q2)
h12 = np.dot(q1,y)
y = y - h12*q1
h22 = np.dot(q2,y)
y = y - h22*q2
h32 = np.linalg.norm(y)
q3 = y / h32

La descomposición quedaría:

$A\,Q_2 = Q_3\,H_2$

In [7]:
Q = np.zeros((3,3))
Q[:,0] = q1
Q[:,1] = q2
Q[:,2] = q3

H = np.array([[h11,h12],[h21,h22],[0.,h32]])

L = A@Q[:,:2]
R = Q@H

#print(L)
#print(R)

El sistema equivalente es:

$H\,c = b_e$

$H\,\begin{bmatrix}c_1 \\ c_2\end{bmatrix} = \begin{bmatrix}\lVert b \rVert \\ 0 \\ 0\end{bmatrix}$

$Qh,Rh = qr(H)$

$Rh\,c = Qh^*\,b_e$

In [8]:
be = np.array([np.linalg.norm(b),0.,0.])
Qh,Rh = np.linalg.qr(H,mode="reduced")
bh = np.dot(Qh.T,be)
c = np.linalg.solve(Rh,bh)
c

array([0.29921072, 0.23839316])

In [9]:
xa = c[0]*q1 + c[1]*q2

np.linalg.norm(b - np.dot(A,xa))

0.6041220933301769